In [90]:
import gmaps
from api_keys import gkey
import gmaps.geojson_geometries
import requests
import json
from matplotlib.cm import inferno
from matplotlib.colors import to_hex
import pandas as pd

gmaps.configure(api_key=gkey)

In [2]:
zip_geojson = requests.get('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json').content
zip_geojson = json.loads(zip_geojson)

In [79]:
restaraunt_df = pd.read_csv('../data/rawdata/restaurant_count_2.csv')
restaraunt_df.head()

,lat,lng,# Restaurants,zipcode
0,29.756845,-95.365652,60,77002.0
1,29.724893,-95.363752,10,77004.0
2,29.749778,-95.345885,41,77003.0
3,29.740970,-95.391301,60,77006.0
4,29.718435,-95.423555,60,77005.0


In [60]:
zip_coord_df = pd.read_csv('../data/rawdata/zipcode_lat_lngs.csv')
zip_coord_df = zip_coord_df.rename(index = str, columns = {'LAT':'lat', 'LNG': 'lng', 'ZIP':'zip'})

In [80]:
harris_zips = []

with open('../data/cleandata/County_Zips.json') as json_file:
    data = json.load(json_file)
    harris_zips = data['Harris']

In [81]:
len(harris_zips)

146

In [82]:
for i in harris_zips:
    i = str(i)

In [83]:
list_index = []

for i in range(len(zip_geojson['features'])):
    if zip_geojson['features'][i]['properties']['ZCTA5CE10'] not in str(harris_zips):
        list_index.append(i)
        
for index in sorted(list_index, reverse=True): 
    del zip_geojson['features'][index]
len(zip_geojson['features'])

135

In [95]:
# We will need to scale the GINI values to lie between 0 and 1
min_gini = min(harris_zips)
max_gini = max(harris_zips)
gini_range = max_gini - min_gini

# def calculate_color(gini):
#     """
#     Convert the GINI coefficient to a color
#     """
#     # make gini a number between 0 and 1
#     normalized_gini = (gini - min_gini) / gini_range

#     # invert gini so that high inequality gives dark color
#     inverse_gini = 1.0 - normalized_gini

#     # transform the gini coefficient to a matplotlib color
#     mpl_color = viridis(inverse_gini)

#     # transform from a matplotlib color to a valid CSS color
#     gmaps_color = to_hex(mpl_color, keep_alpha=False)

#     return gmaps_color

def calculate_colors(input_list):
    _min = 0
    _max = 60
    _range = _max - _min
    
    result = []
    
    for i in input_list:
        norm = (i - _min) / _range
        
        inverse = 1.0 - norm
        mpl_color = inferno(inverse)
        gmaps_color = to_hex(mpl_color, keep_alpha=False)
        
        result.append(gmaps_color)
        
    return result

In [96]:
colors = calculate_colors(restaraunt_df['# Restaurants'])

In [15]:
len(zip_geojson['features'])

135

In [16]:
# colors = []
# index = 0
# for i in range(len(zip_geojson['features'])):
# #     country_name = feature['properties']['name']
#     try:
#         zipcode = int(zip_geojson['features'][i]['properties']['ZCTA5CE10'])
#         color = calculate_color(zipcode)
#     except KeyError:
#         # no GINI for that country: return default color
#         color = (0, 0, 0, 0.3)
#     colors.append(color)
#     index += 1



In [97]:
fig = gmaps.figure(map_type = 'HYBRID')

gini_layer = gmaps.geojson_layer(
    zip_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
zip_layer = gmaps.geojson_layer(zip_geojson)
restaraunt_layer = gmaps.heatmap_layer(restaraunt_df[['lat', 'lng']], weights = restaraunt_df['# Restaurants'])
# fig.add_layer(restaraunt_layer)
fig.add_layer(zip_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))